In [1]:
import pandas as pd
import numpy as np
import localcider
from localcider.sequenceParameters import SequenceParameters

In [2]:
three_to_one = {
    "Ala": "A", "Arg": "R", "Asn": "N", "Asp": "D",
    "Cys": "C", "Gln": "Q", "Glu": "E", "Gly": "G",
    "His": "H", "Ile": "I", "Leu": "L", "Lys": "K",
    "Met": "M", "Phe": "F", "Pro": "P", "Ser": "S",
    "Thr": "T", "Trp": "W", "Tyr": "Y", "Val": "V",
    "Sec": "U", "Pyl": "O"
}

In [4]:
#variant_df = pd.read_csv("variants.csv") # for clinvar data
variant_df = pd.read_csv("gnomad_mutated_full.csv")
protein_db = pd.read_csv("prots.csv")  
variant_df.head()

C:\Users\anvit\AppData\Local\Temp\ipykernel_13988\3685311382.py:2: DtypeWarning: Columns (4,60) have mixed types. Specify dtype option on import or set low_memory=False.
  variant_df = pd.read_csv("gnomad_mutated_full.csv")


,Gene,UniProtKB,ProteinHGVS,Variant ID,Chromosome,Position,rsIDs,Reference,Alternate,Transcript,...,Homozygote East Asian,Hemizygote East Asian,AC African/African American,AN African/African American,Homozygote African/African American,Hemizygote African/African American,Used Sequence,Original Sequence,Mutated Sequence,Error
0,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949907-G-C,4,76949907,rs751652079,G,C,ENST00000264893,...,0,0,0,28284,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MPVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
1,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949908-C-A,4,76949908,NaN,C,A,ENST00000264893,...,0,0,0,28264,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MDVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
2,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949908-C-T,4,76949908,NaN,C,T,ENST00000264893,...,0,0,0,28264,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MVVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
3,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949910-G-T,4,76949910,NaN,G,T,ENST00000264893,...,0,0,0,28248,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MALAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
4,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949910-G-A,4,76949910,rs1387288530,G,A,ENST00000264893,...,0,0,0,28248,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MAMAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN


In [5]:
pd.set_option('display.max_columns', None)
# only rename the columns below for gnomad data, not for clinvar
variant_df = variant_df.rename(columns={"UniProtKB": "UniProtID", "Previous AA": "MutatedFrom", "AA Position": "ProteinPosition", "New AA": "MutatedTo", "Original Sequence": "UnmutatedSeq", "Mutated Sequence": "MutatedSeq"})
variant_df.head()

,Gene,UniProtID,ProteinHGVS,Variant ID,Chromosome,Position,rsIDs,Reference,Alternate,Transcript,Protein Consequence,MutatedFrom,ProteinPosition,MutatedTo,Consequence,Flags,Exome AC,Exome AN,Exome AF,Exome AC Hom,Exome AC Hemi,AC remaining,AN remaining,Homozygote remaining,Hemizygote remaining,AC Admixed American,AN Admixed American,Homozygote Admixed American,Hemizygote Admixed American,AC European (Finnish),AN European (Finnish),Homozygote European (Finnish),Hemizygote European (Finnish),AC Middle Eastern,AN Middle Eastern,Homozygote Middle Eastern,Hemizygote Middle Eastern,AC European (non-Finnish),AN European (non-Finnish),Homozygote European (non-Finnish),Hemizygote European (non-Finnish),AC South Asian,AN South Asian,Homozygote South Asian,Hemizygote South Asian,AC Ashkenazi Jewish,AN Ashkenazi Jewish,Homozygote Ashkenazi Jewish,Hemizygote Ashkenazi Jewish,AC East Asian,AN East Asian,Homozygote East Asian,Hemizygote East Asian,AC African/African American,AN African/African American,Homozygote African/African American,Hemizygote African/African American,Used Sequence,UnmutatedSeq,MutatedSeq,Error
0,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949907-G-C,4,76949907,rs751652079,G,C,ENST00000264893,p.Ala2Pro,Ala,2,Pro,missense_variant,NaN,57,1378416,4.135181e-05,0,0,0,57204,0,0,0,35728,0,0,0,40604,0,0,0,4198,0,0,57,1078404,0,0,0,77374,0,0,0,24022,0,0,0,32598,0,0,0,28284,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MPVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
1,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949908-C-A,4,76949908,NaN,C,A,ENST00000264893,p.Ala2Asp,Ala,2,Asp,missense_variant,NaN,1,1378100,7.256367e-07,0,0,0,57190,0,0,0,35672,0,0,0,40680,0,0,0,4194,0,0,0,1078228,0,0,1,77330,0,0,0,23982,0,0,0,32560,0,0,0,28264,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MDVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
2,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949908-C-T,4,76949908,NaN,C,T,ENST00000264893,p.Ala2Val,Ala,2,Val,missense_variant,NaN,9,1378100,6.530731e-06,0,0,1,57190,0,0,0,35672,0,0,0,40680,0,0,0,4194,0,0,7,1078228,0,0,0,77330,0,0,0,23982,0,0,1,32560,0,0,0,28264,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MVVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
3,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949910-G-T,4,76949910,NaN,G,T,ENST00000264893,p.Val3Leu,Val,3,Leu,missense_variant,NaN,2,1378764,1.450575e-06,0,0,0,57204,0,0,0,35738,0,0,1,40816,0,0,0,4196,0,0,1,1078560,0,0,0,77346,0,0,0,24018,0,0,0,32638,0,0,0,28248,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MALAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN
4,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949910-G-A,4,76949910,rs1387288530,G,A,ENST00000264893,p.Val3Met,Val,3,Met,missense_variant,NaN,2,1378764,1.450575e-06,0,0,0,57204,0,0,2,35738,0,0,0,40816,0,0,0,4196,0,0,0,1078560,0,0,0,77346,0,0,0,24018,0,0,0,32638,0,0,0,28248,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,MAMAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN


In [7]:
merged_df = variant_df.merge(protein_db, left_on="UniProtID", right_on="uniprot", how="left", suffixes=('', '_protein'))
merged_df = merged_df.drop(columns=["uniprot", "uniprot_sequence", "UnmutatedSeq"]) 
merged_df = merged_df.rename(columns={'MutatedSeq': 'sequence'})

In [8]:
merged_df.head()

,Gene,UniProtID,ProteinHGVS,Variant ID,Chromosome,Position,rsIDs,Reference,Alternate,Transcript,Protein Consequence,MutatedFrom,ProteinPosition,MutatedTo,Consequence,Flags,Exome AC,Exome AN,Exome AF,Exome AC Hom,Exome AC Hemi,AC remaining,AN remaining,Homozygote remaining,Hemizygote remaining,AC Admixed American,AN Admixed American,Homozygote Admixed American,Hemizygote Admixed American,AC European (Finnish),AN European (Finnish),Homozygote European (Finnish),Hemizygote European (Finnish),AC Middle Eastern,AN Middle Eastern,Homozygote Middle Eastern,Hemizygote Middle Eastern,AC European (non-Finnish),AN European (non-Finnish),Homozygote European (non-Finnish),Hemizygote European (non-Finnish),AC South Asian,AN South Asian,Homozygote South Asian,Hemizygote South Asian,AC Ashkenazi Jewish,AN Ashkenazi Jewish,Homozygote Ashkenazi Jewish,Hemizygote Ashkenazi Jewish,AC East Asian,AN East Asian,Homozygote East Asian,Hemizygote East Asian,AC African/African American,AN African/African American,Homozygote African/African American,Hemizygote African/African American,Used Sequence,sequence,Error,UNIQUE,ID,Description,HEK293-RIC,HuH7-IC,HeLa-RNPxl,HeLa-IC,HeLa-RBDmap,K562-serIC-chr,K562-serIC,Enzyme,Metabolism,"""Metabolic.Enzyme""",uniprot name,protein_name,gene_name,function,Region,region_start,region_end,disordered_subsequence,MS-identified tryptic peptide (N-link),rbdpep_start,rbdpep_stop,domain,enzyme,LysC/ArgC proteolytic fragment (RBDpep),fragmentStart,fragmentStop,protein_frac_A,protein_frac_C,protein_frac_D,protein_frac_E,protein_frac_F,protein_frac_G,protein_frac_H,protein_frac_I,protein_frac_K,protein_frac_L,protein_frac_M,protein_frac_N,protein_frac_P,protein_frac_Q,protein_frac_R,protein_frac_S,protein_frac_T,protein_frac_V,protein_frac_W,protein_frac_Y,disordered_frac_A,disordered_frac_C,disordered_frac_D,disordered_frac_E,disordered_frac_F,disordered_frac_G,disordered_frac_H,disordered_frac_I,disordered_frac_K,disordered_frac_L,disordered_frac_M,disordered_frac_N,disordered_frac_P,disordered_frac_Q,disordered_frac_R,disordered_frac_S,disordered_frac_T,disordered_frac_V,disordered_frac_W,disordered_frac_Y,protein_frac_charged,protein_frac_aromatic,protein_frac_hydrophobic,protein_frac_polar,protein_frac_g,protein_frac_p,disordered_frac_charged,disordered_frac_aromatic,disordered_frac_hydrophobic,disordered_frac_polar,disordered_frac_g,disordered_frac_p,ncpr,mean_hydropathy,fcr,protein_frac_ILMV,protein_frac_RK,protein_frac_DE,protein_frac_GS,protein_frac_FWY,protein_frac_FWYILV,protein_frac_FWYR,protein_frac_RQ,protein_frac_KQ,protein_frac_NQ,protein_frac_YF,protein_frac_FW,protein_frac_SG,protein_R_over_RK,protein_D_over_DE,protein_S_over_SG,protein_N_over_NQ,protein_Y_over_YF,protein_F_over_FW,protein_R_over_RQ,protein_K_over_KQ,protein_FWY_over_FWYILV,protein_FWY_over_FWYR,disordered_frac_ILMV,disordered_frac_RK,disordered_frac_DE,disordered_frac_GS,disordered_frac_FWY,disordered_frac_FWYILV,disordered_frac_FWYR,disordered_frac_RQ,disordered_frac_KQ,disordered_frac_NQ,disordered_frac_YF,disordered_frac_FW,disordered_frac_SG,disordered_R_over_RK,disordered_D_over_DE,disordered_S_over_SG,disordered_N_over_NQ,disordered_Y_over_YF,disordered_F_over_FW,disordered_R_over_RQ,disordered_K_over_KQ,disordered_FWY_over_FWYILV,disordered_FWY_over_FWYR,From,Entry,Post-translational modification
0,SEPTIN11,Q9NVA2,"NP_001293076.1,NP_060713.1",4-76949907-G-C,4,76949907,rs751652079,G,C,ENST00000264893,p.Ala2Pro,Ala,2,Pro,missense_variant,NaN,57,1378416,0.000041,0,0,0,57204,0,0,0,35728,0,0,0,40604,0,0,0,4198,0,0,57,1078404,0,0,0,77374,0,0,0,24022,0,0,0,32598,0,0,0,28284,0,0,NP: NP_060713.1 septin-11 isoform 2 [Homo sapi...,MPVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILC...,NaN,SEPT11,ENSG00000138758,septin 11,no,no,no,no,YES,no,no,0.0,0.0,0.0,SEP11_HUMAN,Septin-11,SEPTIN11 SEPT11,FUNCTION: Filament-forming cytoskeletal GTPase...,"REGION 48..55; /note=""G1 motif""; /evidence=""EC...",398.0,429.0,KKAAAQLLQSQAQQSGAQQTKKDKDKKNASFT,AAAQLLQSQAQQ

In [10]:
alphamissense = pd.read_csv("missense.tsv", sep="\t", skiprows=3) # download from Google Cloud
# https://console.cloud.google.com/storage/browser/_details/dm_alphamissense/AlphaMissense_aa_substitutions.tsv.gz;tab=live_object
alphamissense.head()

,uniprot_id,protein_variant,am_pathogenicity,am_class
0,A0A024R1R8,M1A,0.4673,ambiguous
1,A0A024R1R8,M1C,0.3828,ambiguous
2,A0A024R1R8,M1D,0.8267,pathogenic
3,A0A024R1R8,M1E,0.5236,ambiguous
4,A0A024R1R8,M1F,0.2753,benign


In [11]:
merged_df['protein_variant'] = (
    merged_df['MutatedFrom'].map(three_to_one) +
    merged_df['ProteinPosition'].astype(str) +
    merged_df['MutatedTo'].map(three_to_one)
)

merged_df = merged_df.merge(
    alphamissense[['uniprot_id', 'protein_variant', 'am_pathogenicity']],
    how='left',
    left_on=['UniProtID', 'protein_variant'],
    right_on=['uniprot_id', 'protein_variant']
)
merged_df = merged_df.rename(columns={'am_pathogenicity': 'alpha_missense'})
merged_df = merged_df.drop(columns='uniprot_id')
merged_df[['UniProtID', 'protein_variant', 'alpha_missense']].head()

,UniProtID,protein_variant,alpha_missense
0,Q9NVA2,A2P,0.2074
1,Q9NVA2,A2P,0.2074
2,Q9NVA2,A2P,0.2074
3,Q9NVA2,A2P,0.2074
4,Q9NVA2,A2P,0.2074


In [12]:
merged_df[['UniProtID', 'protein_variant', 'alpha_missense']].head(50)

,UniProtID,protein_variant,alpha_missense
0,Q9NVA2,A2P,0.2074
1,Q9NVA2,A2P,0.2074
2,Q9NVA2,A2P,0.2074
3,Q9NVA2,A2P,0.2074
4,Q9NVA2,A2P,0.2074
5,Q9NVA2,A2D,0.3795
6,Q9NVA2,A2D,0.3795
7,Q9NVA2,A2D,0.3795
8,Q9NVA2,A2D,0.3795
9,Q9NVA2,A2D,0.3795


In [ ]:
# Generate sequence objects for each sequence
sequences = list(merged_df["sequence"])
sequence_objects = []
i = 0
for sequence in sequences:
    sequence_objects.append(SequenceParameters(sequence))
    i = i + 1
    if (i % 1000 == 0):
        print(i)

In [ ]:
# List of functions that are to be applied
cider_functions = [
    "get_FCR",
    "get_NCPR",
    "get_isoelectric_point",
    "get_molecular_weight",
    "get_countNeg",
    "get_countPos",
    "get_countNeut",
    "get_fraction_negative",
    "get_fraction_positive",
    "get_fraction_expanding",
    "get_amino_acid_fractions",
    "get_fraction_disorder_promoting",
    "get_mean_net_charge",
    "get_mean_hydropathy",
    "get_uversky_hydropathy",
    "get_PPII_propensity",
]

In [ ]:
# Apply the functions
outputs = [
    [getattr(seq, func)() for seq in sequence_objects]
    for func in cider_functions
]

In [ ]:
# Add the outputs to the dataframe
for func_name, values in zip(cider_functions, outputs):
    merged_df[func_name.replace("get_", "")] = values

In [ ]:
# Specify amino acid fractions to be calculated

aa_list = list("ACDEFGHIKLMNPQRSTVWY")
individual_fractions = {f"Fraction_{aa}": (aa, "ACDEFGHIKLMNPQRSTVWY") for aa in aa_list}

composite_fractions = {
    "Fraction_ILMV": ("ILMV", "ACDEFGHIKLMNPQRSTVWY"),
    "Fraction_RK": ("RK", "ACDEFGHIKLMNPQRSTVWY"),
    "Fraction_DE": ("DE", "ACDEFGHIKLMNPQRSTVWY"),
    "Fraction_GS": ("GS", "ACDEFGHIKLMNPQRSTVWY"),
    "Fraction_FWY": ("FWY", "ACDEFGHIKLMNPQRSTVWY"),
    "Fraction_R_over_RK": ("R", "RK"),
    "Fraction_D_over_DE": ("D", "DE"),
    "Fraction_S_over_SG": ("S", "SG"),
    "Fraction_N_over_NQ": ("N", "NQ"),
    "Fraction_Y_over_YF": ("Y", "YF"),
    "Fraction_F_over_FW": ("F", "FW"),
    "Fraction_Y_over_YW": ("Y", "YW"),
    "Fraction_R_over_RQ": ("R", "RQ"),
    "Fraction_K_over_KQ": ("K", "KQ"),
    "Fraction_FWY_over_FWYILV": ("FWY", "FWYILV"),
    "Fraction_FWY_over_FWYR": ("FWY", "FWYR")
}

fractions = {**individual_fractions, **composite_fractions}

In [ ]:
# Function to compute the fraction
def compute_fraction(seq, numerator_set, denominator_set):
    if pd.isna(seq) or not seq:
        return np.nan
    seq = seq.upper()
    num = sum(1 for aa in seq if aa in numerator_set)
    denom = sum(1 for aa in seq if aa in denominator_set)
    return num / denom if denom > 0 else np.nan

In [ ]:
for name, (num_set, denom_set) in fractions.items():
    merged_df[name] = merged_df["sequence"].apply(lambda seq: compute_fraction(seq, num_set, denom_set))
merged_df.to_csv("variant_table_updated.csv", index=False)

In [ ]:
renaming_dict = {
    col: col.replace('_Map', '_Present')
    for col in merged_df.columns if col.endswith('_Map')
}
merged_df = merged_df.rename(columns=renaming_dict)
map_columns = [col for col in merged_df.columns if col.endswith('_Present')]
for col in map_columns:
    merged_df[col] = merged_df.apply(
        lambda row: (
            row[col][int(row['position']) - 1]
            if pd.notna(row[col]) and pd.notna(row['position']) and 0 <= (int(row['position']) - 1) < len(row[col])
            else None
        ),
        axis=1
    )